# FastAPI

Dans ce tutoriel, on explore le développement d'une API en utilisant FastAPI afin de déployer un modèle de classification d'images.

FastAPI est un framework qui sert à développer des API REST en Python. Il autorise la programmation asynchrone.

On commence notre code par importer la classe ``fastapi.FastAPI`` et définir notre application comme une instance de cette classe.

Deux notions importantes qui permettent à FastAPI de traiter les requêtes sont: le chemin et l'opération. 
- Le chemin fait référence à la dernière partie de l'URL à partir du premier "/". Par exemple, dans l'URL "https://www.sspcloud.fr/formation", le chemin est ```/formation```.

- l'opération est une méthode HTTP: POST (création de données), GET (lecture), PUT (modifier), DELETE (supprimer)...

Dans le protocole HTTP, on communique avec chaque chemin en utilisant une ou plusieurs de ces opérations. Ainsi, dans notre API, on va définir un path et une opération pour chaque fonction qu'on va proposer.

Pour ce faire, on précède chaque fonction par l'expression suivante: ```@app_name.operation(path)```.

- Dans le code suivant, la fonction **root** sert comme un accueil pour l'API et elle retourne un message d'accueil. Elle est précédée par ```@app.get('/')```. Cette expression indique à FastAPI que la fonction "root" est chargée des requêtes qui vont au chemin "/", en utilisant l'opération "get".

- La fonction **predict** prend en entrée un fichier image et donne en retour la classe de l'image ainsi que la probabilité de la prédiction. On utilise la syntaxe ``async`` et ``await`` pour qu'elle soit asynchrone. Cela indique à Python d'éxecuter d'autres tâches en attendant que les données soient envoyées du client au serveur à travers le réseau. Cette méthode permet au serveur d'optimiser son temps de réponse.

- On définit également une fonction **details** sur le chemin ``/model/{info}`` et en utilisant l'opération ``get`` pour afficher des informations sur le modèle. Le paramètre ``info`` est appelé paramètre de chemin et il est à préciser par le client dans sa requête. Par exemple, le chemin ``/model/accuracy`` retourne la précision du modèle. 
On ajoute aussi un paramètre de requête appelé ``n``. Ce paramètre est un entier qui permet de préciser le nombre de chiffres après la virgule dans la précision. On définit ce paramètre dans la requête comme dans l'exemple suivant: ``model/accuracy?n=1``. Si n n'est pas de type entier, le serveur nous affichera un erreur.  

Le notebook ``examples.ipynb`` montre les résultats de quelques requêtes.

In [ ]:
#install dependencies
!pip install fastapi
!pip install uvicorn
!pip install python-multipart
!pip install boto3

In [14]:
from io import BytesIO
from fastapi import FastAPI, File, UploadFile
from utils import load_device, import_model, predict, is_image_file
from PIL import Image

 
app = FastAPI()

def read_image(file):
    img = Image.open(BytesIO(file))
    return img

device = load_device()
model = import_model(bucket="mbenxsalha", key="diffusion/state_dict.pickle", device=device)

#url: localhost:8000
@app.get("/")
def root():
    return {"message": "Welcome to Image Classification FastAPI"}

#url: localhost: localhost:8000/model
@app.get("/model/{info}")
def details(info:str, n:int=2):
    accuracy = 99.2511111
    if info == 'model':
        return {'model': 'ResNet18'}
    elif info == 'dataset':
        return {'dataset url': "https://www.kaggle.com/datasets/rhammell/ships-in-satellite-imagery"}
    elif info == 'accuracy':
        formatted_accuracy = int((10**n)*accuracy)/(10**n)
        return {'accuracy': '{}'.format(formatted_accuracy)}
    else:    
        return '{} is not available'.format(info)

#url: localhost:8000/predict
@app.post("/predict")
async def predict_api(file: UploadFile = File(...)):
    if not is_image_file(file.filename):
        return "file must have image format"
    img = read_image(await file.read())
    preds = predict(img, model, device)
    return preds





**Pour lancer l'API, on utilise "uvicorn" qui permet d'exécuter un code asynchrone sur Python.**
Dans le code suivant, ``main`` fait référence au fichier main.py qui contient le même code que la cellule précédente.

La tag ``--reload`` permet de relancer l'API automatiquement si le code est modifié.

In [16]:
!uvicorn main:app --reload

INFO:     Will watch for changes in these directories: ['/home/onyxia/work']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [415] using StatReload
INFO:     Started server process [417]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [417]
